In [1]:
#this notebook is for getting results for each year independentally with all the languages

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import joblib
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics.pairwise import manhattan_distances
import numpy as np
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('dfV2_lite.csv')

In [4]:
df=df.reset_index()
df.drop('index',inplace=True,axis=1)
df.head()

,year,username,language,obfuscated,file_length
0,2008,bhamrick,cpp,0,900
1,2008,bhamrick,cpp,0,3154
2,2008,Davidsu,cpp,0,838
3,2008,matkk,cpp,0,2140
4,2008,Sorokin,cpp,0,2497


In [5]:
#4.17 loss
X,y=joblib.load('struct-char-tan-BIG-500.joblib')

In [6]:
def get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=-1,lang=-1):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]
    #else all languages will be inlcuded
    if lang!=-1:
        df=df[df['language']==lang]
    #else both will be included
    if obfuscated!=-1:
        if obfuscated:
            df=df[df['obfuscated']==1]
        elif obfuscated==0:
            df=df[df['obfuscated']==0]
    df=df[df['year']==year]
    unique_users=df.groupby(['year','username']).count()[lambda x: x > train_sample].dropna().reset_index()['username'].unique()
    for uu in unique_users:
        code_samples=df[df['username']==uu].index.tolist()
        sampled_codes=np.random.choice(code_samples,train_sample+test_sample,replace=False)
        for code_sample in sampled_codes[:-1]:
            X_train.append(X[int(code_sample)])
            y_train.append(uu)
        
        X_test.append(X[sampled_codes[-1]])
        y_test.append(uu)
    return X_train,y_train,X_test,y_test

In [7]:
def top_k_accuracy(y_probas,y_test,neigh,k):
    cor_pred=0
    top_k_predictions = np.argsort(y_probas, axis = 1)[:,-k:]
    top_class = neigh.classes_[top_k_predictions]
    for i in range(len(y_test)):
        if y_test[i] in top_class[i]:
            cor_pred+=1
    return cor_pred/len(y_test)

# Mixed setting all languages

In [8]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in tqdm(range(1,11)):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1)
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

  0%|          | 0/10 [00:00<?, ?it/s]

Year: 2008


 10%|█         | 1/10 [00:20<03:02, 20.29s/it]

Accuracy: 0.8580583146427427


 20%|██        | 2/10 [00:47<02:59, 22.49s/it]

...

 30%|███       | 3/10 [1:15:21<2:54:31, 1495.87s/it]

Accuracy: 0.8068763102725367


 40%|████      | 4/10 [1:41:09<2:31:09, 1511.55s/it]

Accuracy: 0.8060796645702306


MemoryError: Unable to allocate array with shape (23850, 23850) and data type float64

In [8]:
years=[2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in tqdm(range(1,11)):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1)
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

  0%|          | 0/10 [00:00<?, ?it/s]

Year: 2019


 10%|█         | 1/10 [19:03<2:51:30, 1143.44s/it]

Accuracy: 0.8070020964360587


 20%|██        | 2/10 [37:44<2:31:34, 1136.84s/it]

...

Accuracy: 0.8127463312368973


 90%|█████████ | 9/10 [2:48:50<18:42, 1122.50s/it]

Accuracy: 0.8122851153039832


100%|██████████| 10/10 [3:06:46<00:00, 1120.65s/it]

Accuracy: 0.8068343815513627
Unique authors: 23850
Mean Acc: 0.8087798742138365
Std Acc: 0.002575747923094415


In [9]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1)
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9137039431157078
Accuracy: 0.9046541693600517
Accuracy: 0.9140271493212669
Accuracy: 0.9095022624434389
Accuracy: 0.907563025210084
Accuracy: 0.916289592760181
Accuracy: 0.9088558500323206
Accuracy: 0.9059469941822883
Accuracy: 0.9120879120879121
Accuracy: 0.9059469941822883
Unique authors: 3094
Mean Acc: 0.909857789269554
Std Acc: 0.0037759836707230055
Year: 2009
Accuracy: 0.8858950328022493
Accuracy: 0.883552014995314
Accuracy: 0.8830834114339269
Accuracy: 0.8880037488284911
Accuracy: 0.8809746954076851
Accuracy: 0.8830834114339269
Accuracy: 0.8814432989690721
Accuracy: 0.887535145267104
Accuracy: 0.8809746954076851
Accuracy: 0.8812089971883786
Unique authors: 4268
Mean Acc: 0.8835754451733834
Std Acc: 0.002546139242591839
Year: 2010
Accuracy: 0.9023874920769068
Accuracy: 0.9015423621381787
Accuracy: 0.9070357067399113
Accuracy: 0.904077751954363
Accuracy: 0.9009085146841327
Accuracy: 0.9074582717092753
Accuracy: 0.9006972321994506
Accuracy: 0.904500316923727
A

In [10]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1)
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.922509225092251
Accuracy: 0.924149241492415
Accuracy: 0.9339893398933989
Accuracy: 0.9286592865928659
Accuracy: 0.9360393603936039
Accuracy: 0.9220992209922099
Accuracy: 0.932759327593276
Accuracy: 0.931119311193112
Accuracy: 0.922509225092251
Accuracy: 0.9253792537925379
Unique authors: 2439
Mean Acc: 0.9279212792127922
Std Acc: 0.0050007055640801336
Year: 2009
Accuracy: 0.9068169323804288
Accuracy: 0.9073666849917537
Accuracy: 0.9169873556899395
Accuracy: 0.9076415612974161
Accuracy: 0.9098405717427158
Accuracy: 0.9054425508521166
Accuracy: 0.9103903243540407
Accuracy: 0.9095656954370533
Accuracy: 0.9145134689389774
Accuracy: 0.9037932930181418
Unique authors: 3638
Mean Acc: 0.9092358438702585
Std Acc: 0.003816328618319646
Year: 2010
Accuracy: 0.920139755428001
Accuracy: 0.9273770900923384
Accuracy: 0.9276266533566259
Accuracy: 0.917144996256551
Accuracy: 0.9263788370351884
Accuracy: 0.9283753431494884
Accuracy: 0.9241327676566009
Accuracy: 0.9343648614923883
A

# Mixed setting language specific

In [11]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9331619537275064
Accuracy: 0.9254498714652957
Accuracy: 0.9074550128534704
Accuracy: 0.910025706940874
Accuracy: 0.9280205655526992
Accuracy: 0.9203084832904884
Accuracy: 0.9074550128534704
Accuracy: 0.9074550128534704
Accuracy: 0.9203084832904884
Accuracy: 0.9254498714652957
Unique authors: 389
Mean Acc: 0.9185089974293058
Std Acc: 0.009200786718855169
Year: 2009
Accuracy: 0.8833652007648184
Accuracy: 0.8814531548757171
Accuracy: 0.9063097514340345
Accuracy: 0.892925430210325
Accuracy: 0.8718929254302104
Accuracy: 0.8623326959847036
Accuracy: 0.8948374760994264
Accuracy: 0.8986615678776291
Accuracy: 0.8814531548757171
Accuracy: 0.8852772466539197
Unique authors: 523
Mean Acc: 0.8858508604206502
Std Acc: 0.01227438152635108
Year: 2010
Accuracy: 0.9243085880640466
Accuracy: 0.9199417758369723
Accuracy: 0.9446870451237264
Accuracy: 0.9461426491994177
Accuracy: 0.9403202328966521
Accuracy: 0.9432314410480349
Accuracy: 0.9490538573508006
Accuracy: 0.9577874818049491


In [12]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9505494505494505
Accuracy: 0.9560439560439561
Accuracy: 0.9478021978021978
Accuracy: 0.9505494505494505
Accuracy: 0.967032967032967
Accuracy: 0.9587912087912088
Accuracy: 0.9368131868131868
Accuracy: 0.9642857142857143
Accuracy: 0.9587912087912088
Accuracy: 0.967032967032967
Unique authors: 364
Mean Acc: 0.9557692307692307
Std Acc: 0.009065934065934079
Year: 2009
Accuracy: 0.9203187250996016
Accuracy: 0.9203187250996016
Accuracy: 0.9342629482071713
Accuracy: 0.9183266932270916
Accuracy: 0.9163346613545816
Accuracy: 0.9023904382470119
Accuracy: 0.9382470119521913
Accuracy: 0.9402390438247012
Accuracy: 0.9163346613545816
Accuracy: 0.9243027888446215
Unique authors: 502
Mean Acc: 0.9231075697211153
Std Acc: 0.01099052926225797
Year: 2010
Accuracy: 0.9629629629629629
Accuracy: 0.9597423510466989
Accuracy: 0.9516908212560387
Accuracy: 0.9677938808373591
Accuracy: 0.961352657004831
Accuracy: 0.9710144927536232
Accuracy: 0.9516908212560387
Accuracy: 0.9549114331723028
A

In [13]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9568627450980393
Accuracy: 0.9490196078431372
Accuracy: 0.9294117647058824
Accuracy: 0.9686274509803922
Accuracy: 0.9529411764705882
Accuracy: 0.9764705882352941
Accuracy: 0.9803921568627451
Accuracy: 0.9529411764705882
Accuracy: 0.9647058823529412
Accuracy: 0.9725490196078431
Unique authors: 255
Mean Acc: 0.960392156862745
Std Acc: 0.014509803921568622
Year: 2009
Accuracy: 0.9390862944162437
Accuracy: 0.9619289340101523
Accuracy: 0.9365482233502538
Accuracy: 0.949238578680203
Accuracy: 0.9390862944162437
Accuracy: 0.949238578680203
Accuracy: 0.949238578680203
Accuracy: 0.9416243654822335
Accuracy: 0.9365482233502538
Accuracy: 0.9416243654822335
Unique authors: 394
Mean Acc: 0.9444162436548224
Std Acc: 0.007567538840545981
Year: 2010
Accuracy: 0.9710144927536232
Accuracy: 0.968944099378882
Accuracy: 0.9668737060041408
Accuracy: 0.9648033126293996
Accuracy: 0.968944099378882
Accuracy: 0.9751552795031055
Accuracy: 0.9772256728778468
Accuracy: 0.979296066252588
Accu

In [14]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8308400460299195
Accuracy: 0.857307249712313
Accuracy: 0.8469505178365938
Accuracy: 0.8411967779056386
Accuracy: 0.8388952819332566
Accuracy: 0.8434982738780207
Accuracy: 0.8365937859608745
Accuracy: 0.8423475258918297
Accuracy: 0.8319907940161104
Accuracy: 0.8273878020713463
Unique authors: 869
Mean Acc: 0.8397008055235903
Std Acc: 0.008282987761992234
Year: 2009
Accuracy: 0.8137931034482758
Accuracy: 0.787192118226601
Accuracy: 0.8
Accuracy: 0.7733990147783252
Accuracy: 0.8068965517241379
Accuracy: 0.8118226600985222
Accuracy: 0.7891625615763547
Accuracy: 0.7901477832512315
Accuracy: 0.8295566502463054
Accuracy: 0.8009852216748768
Unique authors: 1015
Mean Acc: 0.8002955665024629
Std Acc: 0.015288723154394473
Year: 2010
Accuracy: 0.8667279411764706
Accuracy: 0.8777573529411765
Accuracy: 0.8648897058823529
Accuracy: 0.8759191176470589
Accuracy: 0.8777573529411765
Accuracy: 0.8639705882352942
Accuracy: 0.8731617647058824
Accuracy: 0.8786764705882353
Accuracy: 0.8

In [15]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9186602870813397
Accuracy: 0.8935406698564593
Accuracy: 0.9007177033492823
Accuracy: 0.9043062200956937
Accuracy: 0.888755980861244
Accuracy: 0.8923444976076556
Accuracy: 0.90311004784689
Accuracy: 0.895933014354067
Accuracy: 0.9055023923444976
Accuracy: 0.9055023923444976
Unique authors: 836
Mean Acc: 0.9008373205741627
Std Acc: 0.008208391609711851
Year: 2009
Accuracy: 0.86497461928934
Accuracy: 0.8568527918781725
Accuracy: 0.8619289340101522
Accuracy: 0.8791878172588833
Accuracy: 0.8761421319796955
Accuracy: 0.867005076142132
Accuracy: 0.8538071065989847
Accuracy: 0.8375634517766497
Accuracy: 0.86497461928934
Accuracy: 0.8538071065989847
Unique authors: 985
Mean Acc: 0.8616243654822334
Std Acc: 0.011378260625532027
Year: 2010
Accuracy: 0.9086538461538461
Accuracy: 0.9211538461538461
Accuracy: 0.9240384615384616
Accuracy: 0.9153846153846154
Accuracy: 0.9163461538461538
Accuracy: 0.9201923076923076
Accuracy: 0.9105769230769231
Accuracy: 0.9125
Accuracy: 0.931730

In [16]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9161184210526315
Accuracy: 0.9177631578947368
Accuracy: 0.9292763157894737
Accuracy: 0.9194078947368421
Accuracy: 0.9407894736842105
Accuracy: 0.9259868421052632
Accuracy: 0.9243421052631579
Accuracy: 0.9276315789473685
Accuracy: 0.9243421052631579
Accuracy: 0.9342105263157895
Unique authors: 608
Mean Acc: 0.9259868421052632
Std Acc: 0.007169241683455061
Year: 2009
Accuracy: 0.8987341772151899
Accuracy: 0.9253164556962026
Accuracy: 0.9
Accuracy: 0.8962025316455696
Accuracy: 0.9037974683544304
Accuracy: 0.8962025316455696
Accuracy: 0.8886075949367088
Accuracy: 0.9
Accuracy: 0.9
Accuracy: 0.9075949367088607
Unique authors: 790
Mean Acc: 0.9016455696202532
Std Acc: 0.009198796100771742
Year: 2010
Accuracy: 0.9358974358974359
Accuracy: 0.951048951048951
Accuracy: 0.9428904428904429
Accuracy: 0.9522144522144522
Accuracy: 0.9475524475524476
Accuracy: 0.9417249417249417
Accuracy: 0.9475524475524476
Accuracy: 0.9557109557109557
Accuracy: 0.9370629370629371
Accuracy: 0.93

In [ ]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8619777895293496
Accuracy: 0.8656795346377578
Accuracy: 0.8836594394500265
Accuracy: 0.866737176097303
Accuracy: 0.8736118455843469
Accuracy: 0.8672659968270756
Accuracy: 0.8683236382866208
Accuracy: 0.8693812797461661
Accuracy: 0.8714965626652564
Accuracy: 0.8751983077736647
Unique authors: 1891
Mean Acc: 0.8703331570597568
Std Acc: 0.005762932935304183
Year: 2009
Accuracy: 0.8346456692913385
Accuracy: 0.8274874731567645
Accuracy: 0.8239083750894775
Accuracy: 0.8385826771653543
Accuracy: 0.8310665712240516
Accuracy: 0.8242662848962061
Accuracy: 0.8317823908375089
Accuracy: 0.8242662848962061
Accuracy: 0.829277022190408
Accuracy: 0.8428775948460988
Unique authors: 2794
Mean Acc: 0.8308160343593414
Std Acc: 0.006069816882834205
Year: 2010
Accuracy: 0.8702460850111857
Accuracy: 0.87248322147651
Accuracy: 0.8731224033237456
Accuracy: 0.8801534036433365
Accuracy: 0.8775966762543944
Accuracy: 0.880792585490572
Accuracy: 0.8897411313518696
Accuracy: 0.8785554490252477


In [8]:
years=[2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2017
Accuracy: 0.9176771759633522
Accuracy: 0.9194287254109405
Accuracy: 0.9167340339531124
Accuracy: 0.9145782807868499
Accuracy: 0.9250875774723795
Accuracy: 0.9213150094314201
Accuracy: 0.9129614659121531
Accuracy: 0.9174077068175693
Accuracy: 0.9129614659121531
Accuracy: 0.9190245216922662
Unique authors: 7422
Mean Acc: 0.9177175963352197
Std Acc: 0.003578490396008585
Year: 2018
Accuracy: 0.9116077384923282
Accuracy: 0.9112741827885257
Accuracy: 0.9156104069379586
Accuracy: 0.9109406270847231
Accuracy: 0.9162775183455637
Accuracy: 0.9156104069379586
Accuracy: 0.9157215921725594
Accuracy: 0.9150544807649544
Accuracy: 0.914609739826551
Accuracy: 0.9133867022459418
Unique authors: 8994
Mean Acc: 0.9140093395597064
Std Acc: 0.0019418939688343956
Year: 2019
Accuracy: 0.8874377601828423
Accuracy: 0.8810709329850624
Accuracy: 0.8869480042445514
Accuracy: 0.8809893069953473
Accuracy: 0.8827034527793649
Accuracy: 0.8865398742959758
Accuracy: 0.8845808505428129
Accuracy: 0.883029956738

In [9]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9020234291799787
Accuracy: 0.9142705005324814
Accuracy: 0.9222577209797657
Accuracy: 0.9222577209797657
Accuracy: 0.9222577209797657
Accuracy: 0.9217252396166135
Accuracy: 0.922790202342918
Accuracy: 0.9286474973375932
Accuracy: 0.917465388711395
Accuracy: 0.9265175718849841
Unique authors: 1878
Mean Acc: 0.920021299254526
Std Acc: 0.0071113678486886345
Year: 2009
Accuracy: 0.8962025316455696
Accuracy: 0.8922242314647378
Accuracy: 0.8965641952983725
Accuracy: 0.8925858951175407
Accuracy: 0.8864376130198915
Accuracy: 0.8958408679927667
Accuracy: 0.8936708860759494
Accuracy: 0.8936708860759494
Accuracy: 0.8889692585895117
Accuracy: 0.8922242314647378
Unique authors: 2765
Mean Acc: 0.8928390596745027
Std Acc: 0.003039050872845668
Year: 2010
Accuracy: 0.9201736806947228
Accuracy: 0.9195056780227121
Accuracy: 0.9228456913827655
Accuracy: 0.9215096860387442
Accuracy: 0.9195056780227121
Accuracy: 0.9225116900467601
Accuracy: 0.9195056780227121
Accuracy: 0.91850367401469

In [10]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Unique authors: 1547
Mean Acc: 0.934453781512605
Std Acc: 0.004899106741175527
Year: 2009
Unique authors: 2407
Mean Acc: 0.9169090153718322
Std Acc: 0.006922032604788872
Year: 2010
Unique authors: 2530
Mean Acc: 0.9398418972332016
Std Acc: 0.0033201581027667796
Year: 2011
Unique authors: 3786
Mean Acc: 0.9536450079239304
Std Acc: 0.003990476737912862
Year: 2012
Unique authors: 3353
Mean Acc: 0.9457798985982702
Std Acc: 0.004360433494183618
Year: 2013
Unique authors: 4636
Mean Acc: 0.9475194132873167
Std Acc: 0.003654307481491499
Year: 2014
Unique authors: 5218
Mean Acc: 0.9471061709467229
Std Acc: 0.0031232730988828444
Year: 2015
Unique authors: 4643
Mean Acc: 0.9553306052121474
Std Acc: 0.0038228227839472383
Year: 2016
Unique authors: 6575
Mean Acc: 0.9445171102661598
Std Acc: 0.00196286123861922
Year: 2017
Unique authors: 5922
Mean Acc: 0.961870989530564
Std Acc: 0.0023395749763077344
Year: 2018
Unique authors: 7839
Mean Acc: 0.9560147978058424
Std Acc: 0.00283406178057001

## Obfuscated only all languages

In [11]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=1)
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8315573770491803
Accuracy: 0.8295081967213115
Accuracy: 0.8176229508196722
Accuracy: 0.8307377049180328
Accuracy: 0.8139344262295082
Accuracy: 0.8282786885245902
Accuracy: 0.8377049180327869
Accuracy: 0.830327868852459
Accuracy: 0.8463114754098361
Accuracy: 0.8262295081967214
Unique authors: 2440
Mean Acc: 0.8292213114754098
Std Acc: 0.008666942728808415
Year: 2009
Accuracy: 0.7839692561076036
Accuracy: 0.7883612407356574
Accuracy: 0.788086741696404
Accuracy: 0.775185286851496
Accuracy: 0.7828712599505901
Accuracy: 0.7867142465001372
Accuracy: 0.7801262695580565
Accuracy: 0.7784792753225364
Accuracy: 0.7820477628328301
Accuracy: 0.7845182541861103
Unique authors: 3643
Mean Acc: 0.7830359593741422
Std Acc: 0.004029062450048581
Year: 2010
Accuracy: 0.8243639921722113
Accuracy: 0.8233855185909981
Accuracy: 0.825587084148728
Accuracy: 0.826320939334638
Accuracy: 0.8297455968688845
Accuracy: 0.8336594911937377
Accuracy: 0.8295009784735812
Accuracy: 0.8341487279843445


In [12]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=1)
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8803986710963455
Accuracy: 0.86578073089701
Accuracy: 0.8757475083056478
Accuracy: 0.8830564784053156
Accuracy: 0.8803986710963455
Accuracy: 0.8717607973421927
Accuracy: 0.8651162790697674
Accuracy: 0.8704318936877077
Accuracy: 0.8724252491694352
Accuracy: 0.8710963455149502
Unique authors: 1505
Mean Acc: 0.8736212624584716
Std Acc: 0.005836595682657209
Year: 2009
Accuracy: 0.8586138613861386
Accuracy: 0.8510891089108911
Accuracy: 0.8594059405940594
Accuracy: 0.8645544554455445
Accuracy: 0.8621782178217822
Accuracy: 0.8542574257425742
Accuracy: 0.8645544554455445
Accuracy: 0.8613861386138614
Accuracy: 0.8601980198019802
Accuracy: 0.8487128712871287
Unique authors: 2525
Mean Acc: 0.8584950495049505
Std Acc: 0.00517298071118484
Year: 2010
Accuracy: 0.8803508771929824
Accuracy: 0.8842105263157894
Accuracy: 0.8768421052631579
Accuracy: 0.8747368421052631
Accuracy: 0.8736842105263158
Accuracy: 0.872280701754386
Accuracy: 0.8912280701754386
Accuracy: 0.8859649122807017

In [13]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=1)
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Unique authors: 1040
Mean Acc: 0.9050961538461539
Std Acc: 0.01009432217816955
Year: 2009
Unique authors: 1547
Mean Acc: 0.9058177117000646
Std Acc: 0.00647090788188785
Year: 2010
Unique authors: 1768
Mean Acc: 0.9134615384615385
Std Acc: 0.006359035892558924
Year: 2011
Unique authors: 2985
Mean Acc: 0.9398324958123953
Std Acc: 0.003118278399522988
Year: 2012
Unique authors: 1842
Mean Acc: 0.9248099891422366
Std Acc: 0.005219919817181922
Year: 2013
Unique authors: 2993
Mean Acc: 0.9253591713999331
Std Acc: 0.0033082242016669707
Year: 2014
Unique authors: 3301
Mean Acc: 0.9244168433807938
Std Acc: 0.005148617892361312
Year: 2015
Unique authors: 3307
Mean Acc: 0.93689144239492
Std Acc: 0.00278477003201519
Year: 2016
Unique authors: 5785
Mean Acc: 0.9043906655142611
Std Acc: 0.0034228630226459827
Year: 2017
Unique authors: 4141
Mean Acc: 0.9356435643564357
Std Acc: 0.0037070504079538824
Year: 2018
Unique authors: 9163
Mean Acc: 0.9108261486412748
Std Acc: 0.0026307768666021265


## obfuscated only language specific

In [14]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.813953488372093
Accuracy: 0.834625322997416
Accuracy: 0.8307493540051679
Accuracy: 0.8268733850129198
Accuracy: 0.8262273901808785
Accuracy: 0.8404392764857881
Accuracy: 0.8372093023255814
Accuracy: 0.8391472868217055
Accuracy: 0.8320413436692506
Accuracy: 0.8217054263565892
Unique authors: 1548
Mean Acc: 0.830297157622739
Std Acc: 0.007875045301115106
Year: 2009
Accuracy: 0.7798088907353552
Accuracy: 0.7931034482758621
Accuracy: 0.7802243456584961
Accuracy: 0.8014125467386789
Accuracy: 0.7877025342750311
Accuracy: 0.7789779808890736
Accuracy: 0.790610718737017
Accuracy: 0.7972579975072704
Accuracy: 0.7877025342750311
Accuracy: 0.7897798088907354
Unique authors: 2407
Mean Acc: 0.788658080598255
Std Acc: 0.007116416622265097
Year: 2010
Accuracy: 0.843677269200931
Accuracy: 0.8483320403413499
Accuracy: 0.8487199379363848
Accuracy: 0.8595810705973623
Accuracy: 0.8487199379363848
Accuracy: 0.8456167571761055
Accuracy: 0.8425135764158262
Accuracy: 0.8429014740108611
A

In [15]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8731203007518797
Accuracy: 0.8768796992481203
Accuracy: 0.8853383458646616
Accuracy: 0.8543233082706767
Accuracy: 0.881578947368421
Accuracy: 0.8825187969924813
Accuracy: 0.8740601503759399
Accuracy: 0.8740601503759399
Accuracy: 0.8787593984962406
Accuracy: 0.8693609022556391
Unique authors: 1064
Mean Acc: 0.875
Std Acc: 0.008300527130007371
Year: 2009
Accuracy: 0.8500576701268743
Accuracy: 0.8604382929642446
Accuracy: 0.8581314878892734
Accuracy: 0.8650519031141869
Accuracy: 0.8529411764705882
Accuracy: 0.8598615916955017
Accuracy: 0.8483275663206459
Accuracy: 0.8529411764705882
Accuracy: 0.8587081891580162
Accuracy: 0.8523644752018454
Unique authors: 1734
Mean Acc: 0.8558823529411764
Std Acc: 0.00504374634465659
Year: 2010
Accuracy: 0.8871595330739299
Accuracy: 0.8921623123957755
Accuracy: 0.9088382434685937
Accuracy: 0.9049471928849361
Accuracy: 0.9127292940522512
Accuracy: 0.9071706503613118
Accuracy: 0.8932740411339634
Accuracy: 0.8977209560867149
Accuracy: 

In [16]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=1,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Unique authors: 807
Mean Acc: 0.9048327137546467
Std Acc: 0.011857123879956077
Year: 2009
Unique authors: 1111
Mean Acc: 0.903150315031503
Std Acc: 0.005595056802131429
Year: 2010
Unique authors: 1094
Mean Acc: 0.9332723948811701
Std Acc: 0.007255259536740195
Year: 2011
Unique authors: 2018
Mean Acc: 0.945143706640238
Std Acc: 0.003927292378805669
Year: 2012
Unique authors: 1258
Mean Acc: 0.9334658187599363
Std Acc: 0.0046899841017488154
Year: 2013
Unique authors: 1831
Mean Acc: 0.9418896777717094
Std Acc: 0.00409575822157349
Year: 2014
Unique authors: 2195
Mean Acc: 0.9381321184510251
Std Acc: 0.006429342091049895
Year: 2015
Unique authors: 2307
Mean Acc: 0.9492414390983962
Std Acc: 0.003766145706154457
Year: 2016
Unique authors: 3314
Mean Acc: 0.9323476161738082
Std Acc: 0.004343106179920834
Year: 2017
Unique authors: 2760
Mean Acc: 0.9499275362318841
Std Acc: 0.003672848086224421
Year: 2018
Unique authors: 5293
Mean Acc: 0.9404307576043831
Std Acc: 0.0024283778987981178
Y

In [17]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=1,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.872093023255814
Accuracy: 0.9108527131782945
Accuracy: 0.8449612403100775
Accuracy: 0.8875968992248062
Accuracy: 0.872093023255814
Accuracy: 0.8798449612403101
Accuracy: 0.875968992248062
Accuracy: 0.8643410852713178
Accuracy: 0.8992248062015504
Accuracy: 0.8837209302325582
Unique authors: 258
Mean Acc: 0.8790697674418604
Std Acc: 0.017316517754797436
Year: 2009
Accuracy: 0.8170426065162907
Accuracy: 0.8395989974937343
Accuracy: 0.7944862155388471
Accuracy: 0.8195488721804511
Accuracy: 0.8370927318295739
Accuracy: 0.8195488721804511
Accuracy: 0.8220551378446115
Accuracy: 0.8095238095238095
Accuracy: 0.8320802005012531
Accuracy: 0.8345864661654135
Unique authors: 399
Mean Acc: 0.8225563909774436
Std Acc: 0.013204952252951862
Year: 2010
Accuracy: 0.9190751445086706
Accuracy: 0.9132947976878613
Accuracy: 0.8766859344894027
Accuracy: 0.8651252408477842
Accuracy: 0.905587668593449
Accuracy: 0.9190751445086706
Accuracy: 0.9344894026974951
Accuracy: 0.9075144508670521
A

In [18]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=1,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9734513274336283
Accuracy: 0.9292035398230089
Accuracy: 0.9557522123893806
Accuracy: 0.9646017699115044
Accuracy: 0.9469026548672567
Accuracy: 0.9292035398230089
Accuracy: 0.9380530973451328
Accuracy: 0.9469026548672567
Accuracy: 0.9646017699115044
Accuracy: 0.9646017699115044
Unique authors: 113
Mean Acc: 0.9513274336283185
Std Acc: 0.014939772580649652
Year: 2009
Accuracy: 0.9022222222222223
Accuracy: 0.9244444444444444
Accuracy: 0.9155555555555556
Accuracy: 0.9155555555555556
Accuracy: 0.9111111111111111
Accuracy: 0.8977777777777778
Accuracy: 0.9066666666666666
Accuracy: 0.9288888888888889
Accuracy: 0.9377777777777778
Accuracy: 0.9022222222222223
Unique authors: 225
Mean Acc: 0.9142222222222222
Std Acc: 0.01226054597700775
Year: 2010
Accuracy: 0.9318181818181818
Accuracy: 0.9545454545454546
Accuracy: 0.9383116883116883
Accuracy: 0.9545454545454546
Accuracy: 0.9383116883116883
Accuracy: 0.9318181818181818
Accuracy: 0.9577922077922078
Accuracy: 0.944805194805194

In [19]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=1,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Unique authors: 39
Mean Acc: 0.9692307692307691
Std Acc: 0.019187986598840738
Year: 2009
Unique authors: 99
Mean Acc: 0.9393939393939394
Std Acc: 0.021188057538790928
Year: 2010
Unique authors: 150
Mean Acc: 0.946
Std Acc: 0.014126413400278062
Year: 2011
Unique authors: 271
Mean Acc: 0.9642066420664206
Std Acc: 0.008582806900083405
Year: 2012
Unique authors: 167
Mean Acc: 0.9610778443113773
Std Acc: 0.01112226085090223
Year: 2013
Unique authors: 332
Mean Acc: 0.947289156626506
Std Acc: 0.010964020917591143
Year: 2014
Unique authors: 384
Mean Acc: 0.9346354166666668
Std Acc: 0.00963541666666667
Year: 2015
Unique authors: 382
Mean Acc: 0.956282722513089
Std Acc: 0.009072106518933764
Year: 2016
Unique authors: 1116
Mean Acc: 0.8975806451612902
Std Acc: 0.008613810049258278
Year: 2017
Unique authors: 670
Mean Acc: 0.9177611940298508
Std Acc: 0.007739637111918901
Year: 2018
Unique authors: 2154
Mean Acc: 0.8891364902506963
Std Acc: 0.004202960184867713
Year: 2019
Unique authors: 

In [20]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=1,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8316831683168316
Accuracy: 0.8333333333333334
Accuracy: 0.8151815181518152
Accuracy: 0.8399339933993399
Accuracy: 0.8168316831683168
Accuracy: 0.8366336633663366
Accuracy: 0.8168316831683168
Accuracy: 0.834983498349835
Accuracy: 0.8333333333333334
Accuracy: 0.8465346534653465
Unique authors: 606
Mean Acc: 0.8305280528052805
Std Acc: 0.010146837715936123
Year: 2009
Accuracy: 0.8124207858048162
Accuracy: 0.7743979721166033
Accuracy: 0.8010139416983524
Accuracy: 0.7997465145754119
Accuracy: 0.8162230671736375
Accuracy: 0.7769328263624842
Accuracy: 0.8098859315589354
Accuracy: 0.8124207858048162
Accuracy: 0.779467680608365
Accuracy: 0.8010139416983524
Unique authors: 789
Mean Acc: 0.7983523447401775
Std Acc: 0.01501192390780002
Year: 2010
Accuracy: 0.8761682242990654
Accuracy: 0.8714953271028038
Accuracy: 0.8808411214953271
Accuracy: 0.8714953271028038
Accuracy: 0.8703271028037384
Accuracy: 0.8598130841121495
Accuracy: 0.8633177570093458
Accuracy: 0.866822429906542
A

In [21]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=1,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8844884488448845
Accuracy: 0.8943894389438944
Accuracy: 0.8745874587458746
Accuracy: 0.900990099009901
Accuracy: 0.900990099009901
Accuracy: 0.8877887788778878
Accuracy: 0.9207920792079208
Accuracy: 0.9141914191419142
Accuracy: 0.8778877887788779
Accuracy: 0.8976897689768977
Unique authors: 303
Mean Acc: 0.8953795379537954
Std Acc: 0.01408355662927516
Year: 2009
Accuracy: 0.8888888888888888
Accuracy: 0.8767676767676768
Accuracy: 0.8787878787878788
Accuracy: 0.9050505050505051
Accuracy: 0.8767676767676768
Accuracy: 0.8868686868686869
Accuracy: 0.8626262626262626
Accuracy: 0.8686868686868687
Accuracy: 0.8707070707070707
Accuracy: 0.8828282828282829
Unique authors: 495
Mean Acc: 0.8797979797979798
Std Acc: 0.011401172573351498
Year: 2010
Accuracy: 0.9382716049382716
Accuracy: 0.9312169312169312
Accuracy: 0.9365079365079365
Accuracy: 0.9171075837742504
Accuracy: 0.9435626102292769
Accuracy: 0.9294532627865961
Accuracy: 0.9118165784832452
Accuracy: 0.9382716049382716


In [22]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=1,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9421965317919075
Accuracy: 0.9421965317919075
Accuracy: 0.9017341040462428
Accuracy: 0.930635838150289
Accuracy: 0.9479768786127167
Accuracy: 0.9364161849710982
Accuracy: 0.9075144508670521
Accuracy: 0.9479768786127167
Accuracy: 0.8959537572254336
Accuracy: 0.9479768786127167
Unique authors: 173
Mean Acc: 0.9300578034682081
Std Acc: 0.019422309383134596
Year: 2009
Accuracy: 0.900355871886121
Accuracy: 0.9323843416370107
Accuracy: 0.9430604982206405
Accuracy: 0.9181494661921709
Accuracy: 0.9217081850533808
Accuracy: 0.9430604982206405
Accuracy: 0.9288256227758007
Accuracy: 0.9323843416370107
Accuracy: 0.9217081850533808
Accuracy: 0.9430604982206405
Unique authors: 281
Mean Acc: 0.9284697508896798
Std Acc: 0.01287548221717119
Year: 2010
Accuracy: 0.9671641791044776
Accuracy: 0.9701492537313433
Accuracy: 0.9582089552238806
Accuracy: 0.9671641791044776
Accuracy: 0.9582089552238806
Accuracy: 0.9611940298507463
Accuracy: 0.9671641791044776
Accuracy: 0.9582089552238806


## non obfuscated all langs

In [23]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=0)
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8969671790610719
Accuracy: 0.8998753635230577
Accuracy: 0.9106771915247196
Accuracy: 0.9052762775238886
Accuracy: 0.8973826339842127
Accuracy: 0.8990444536767761
Accuracy: 0.8994599085999169
Accuracy: 0.9036144578313253
Accuracy: 0.8894889904445368
Accuracy: 0.894058994599086
Unique authors: 2407
Mean Acc: 0.8995845450768591
Std Acc: 0.005623393791581984
Year: 2009
Accuracy: 0.8760399334442596
Accuracy: 0.8729894620077648
Accuracy: 0.8632834165280089
Accuracy: 0.8627287853577371
Accuracy: 0.8718801996672213
Accuracy: 0.8718801996672213
Accuracy: 0.8693843594009983
Accuracy: 0.8691070438158625
Accuracy: 0.8713255684969495
Accuracy: 0.8632834165280089
Unique authors: 3606
Mean Acc: 0.8691902384914032
Std Acc: 0.004381322962947193
Year: 2010
Accuracy: 0.9083969465648855
Accuracy: 0.9089234008949724
Accuracy: 0.9047117662542774
Accuracy: 0.8978678599631482
Accuracy: 0.912345354040537
Accuracy: 0.9176098973414056
Accuracy: 0.9049749934193209
Accuracy: 0.89892076862332

In [24]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=0)
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9368703828072532
Accuracy: 0.9355271994627267
Accuracy: 0.9408999328408327
Accuracy: 0.9281396910678308
Accuracy: 0.9341840161182001
Accuracy: 0.9355271994627267
Accuracy: 0.9308260577568838
Accuracy: 0.9321692411014103
Accuracy: 0.9382135661517798
Accuracy: 0.9321692411014103
Unique authors: 1489
Mean Acc: 0.9344526527871053
Std Acc: 0.0035689268651676116
Year: 2009
Accuracy: 0.9267707082833133
Accuracy: 0.9231692677070829
Accuracy: 0.9243697478991597
Accuracy: 0.9303721488595438
Accuracy: 0.9135654261704682
Accuracy: 0.9287715086034414
Accuracy: 0.9231692677070829
Accuracy: 0.9179671868747499
Accuracy: 0.9207683073229291
Accuracy: 0.9243697478991597
Unique authors: 2499
Mean Acc: 0.923329331732693
Std Acc: 0.004732051380262335
Year: 2010
Accuracy: 0.937793427230047
Accuracy: 0.935054773082942
Accuracy: 0.9315336463223787
Accuracy: 0.9315336463223787
Accuracy: 0.940923317683881
Accuracy: 0.9362284820031299
Accuracy: 0.935054773082942
Accuracy: 0.9389671361502347

In [25]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=0)
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.954368932038835
Accuracy: 0.9611650485436893
Accuracy: 0.945631067961165
Accuracy: 0.9572815533980582
Accuracy: 0.9533980582524272
Accuracy: 0.9572815533980582
Accuracy: 0.9504854368932039
Accuracy: 0.9485436893203884
Accuracy: 0.9495145631067962
Accuracy: 0.9553398058252427
Unique authors: 1030
Mean Acc: 0.9533009708737865
Std Acc: 0.004500709509320096
Year: 2009
Accuracy: 0.9640287769784173
Accuracy: 0.9444081098757358
Accuracy: 0.9594506213211249
Accuracy: 0.9542184434270765
Accuracy: 0.9542184434270765
Accuracy: 0.9581425768476128
Accuracy: 0.9522563767168084
Accuracy: 0.9574885546108568
Accuracy: 0.9535644211903205
Accuracy: 0.9601046435578809
Unique authors: 1529
Mean Acc: 0.955788096795291
Std Acc: 0.005101373446697189
Year: 2010
Accuracy: 0.9475728155339805
Accuracy: 0.948220064724919
Accuracy: 0.9559870550161812
Accuracy: 0.9533980582524272
Accuracy: 0.943042071197411
Accuracy: 0.9533980582524272
Accuracy: 0.9566343042071197
Accuracy: 0.9592233009708738


## non-obfuscated only lang specific

In [26]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=0,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.8912760416666666
Accuracy: 0.8984375
Accuracy: 0.892578125
Accuracy: 0.890625
Accuracy: 0.8912760416666666
Accuracy: 0.90234375
Accuracy: 0.8984375
Accuracy: 0.9114583333333334
Accuracy: 0.8919270833333334
Accuracy: 0.8938802083333334
Unique authors: 1536
Mean Acc: 0.8962239583333333
Std Acc: 0.006299984585105711
Year: 2009
Accuracy: 0.8645833333333334
Accuracy: 0.8670833333333333
Accuracy: 0.8604166666666667
Accuracy: 0.85625
Accuracy: 0.86375
Accuracy: 0.86375
Accuracy: 0.8675
Accuracy: 0.8695833333333334
Accuracy: 0.8633333333333333
Accuracy: 0.8708333333333333
Unique authors: 2400
Mean Acc: 0.8647083333333333
Std Acc: 0.004103902275747926
Year: 2010
Accuracy: 0.9222362337116435
Accuracy: 0.9218158890290038
Accuracy: 0.9264396805380412
Accuracy: 0.9192938209331651
Accuracy: 0.9255989911727617
Accuracy: 0.9319041614123581
Accuracy: 0.9226565783942833
Accuracy: 0.9281210592686002
Accuracy: 0.9293820933165196
Accuracy: 0.9260193358554014
Unique authors: 2379
Mean

In [27]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=0,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9348441926345609
Accuracy: 0.9423984891406988
Accuracy: 0.943342776203966
Accuracy: 0.9405099150141643
Accuracy: 0.9405099150141643
Accuracy: 0.9367327667610954
Accuracy: 0.9423984891406988
Accuracy: 0.9357884796978282
Accuracy: 0.9263456090651558
Accuracy: 0.9320113314447592
Unique authors: 1059
Mean Acc: 0.937488196411709
Std Acc: 0.00515134341718067
Year: 2009
Accuracy: 0.9246813441483198
Accuracy: 0.9275782155272306
Accuracy: 0.9223638470451911
Accuracy: 0.9212050984936269
Accuracy: 0.9339513325608343
Accuracy: 0.9333719582850522
Accuracy: 0.9246813441483198
Accuracy: 0.9183082271147162
Accuracy: 0.9194669756662804
Accuracy: 0.9223638470451911
Unique authors: 1726
Mean Acc: 0.9247972190034762
Std Acc: 0.00510244237689548
Year: 2010
Accuracy: 0.9611772072636193
Accuracy: 0.9505322479649343
Accuracy: 0.9574201628052599
Accuracy: 0.954915466499687
Accuracy: 0.9536631183469004
Accuracy: 0.9580463368816531
Accuracy: 0.9517845961177207
Accuracy: 0.9561678146524734


In [28]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=0,lang='cpp')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9488139825218477
Accuracy: 0.9637952559300874
Accuracy: 0.9438202247191011
Accuracy: 0.9600499375780275
Accuracy: 0.951310861423221
Accuracy: 0.9600499375780275
Accuracy: 0.9588014981273408
Accuracy: 0.9550561797752809
Accuracy: 0.9575530586766542
Accuracy: 0.9538077403245943
Unique authors: 801
Mean Acc: 0.9553058676654181
Std Acc: 0.005715617049817541
Year: 2009
Accuracy: 0.9528985507246377
Accuracy: 0.9456521739130435
Accuracy: 0.9664855072463768
Accuracy: 0.9601449275362319
Accuracy: 0.9565217391304348
Accuracy: 0.9583333333333334
Accuracy: 0.9501811594202898
Accuracy: 0.9492753623188406
Accuracy: 0.9610507246376812
Accuracy: 0.9583333333333334
Unique authors: 1104
Mean Acc: 0.9558876811594204
Std Acc: 0.005995391573685855
Year: 2010
Accuracy: 0.9686847599164927
Accuracy: 0.9801670146137788
Accuracy: 0.9822546972860126
Accuracy: 0.9770354906054279
Accuracy: 0.9718162839248434
Accuracy: 0.9707724425887265
Accuracy: 0.9728601252609603
Accuracy: 0.96868475991649

In [29]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=0,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9572649572649573
Accuracy: 0.9529914529914529
Accuracy: 0.9700854700854701
Accuracy: 0.9444444444444444
Accuracy: 0.9658119658119658
Accuracy: 0.9615384615384616
Accuracy: 0.9529914529914529
Accuracy: 0.9829059829059829
Accuracy: 0.9829059829059829
Accuracy: 0.9700854700854701
Unique authors: 234
Mean Acc: 0.964102564102564
Std Acc: 0.012117475964750274
Year: 2009
Accuracy: 0.9405405405405406
Accuracy: 0.9486486486486486
Accuracy: 0.9540540540540541
Accuracy: 0.927027027027027
Accuracy: 0.9324324324324325
Accuracy: 0.9540540540540541
Accuracy: 0.9459459459459459
Accuracy: 0.9756756756756757
Accuracy: 0.9459459459459459
Accuracy: 0.9486486486486486
Unique authors: 370
Mean Acc: 0.9472972972972972
Std Acc: 0.01257555689095335
Year: 2010
Accuracy: 0.9712918660287081
Accuracy: 0.9784688995215312
Accuracy: 0.9665071770334929
Accuracy: 0.9736842105263158
Accuracy: 0.9760765550239234
Accuracy: 0.9641148325358851
Accuracy: 0.9688995215311005
Accuracy: 0.9736842105263158


In [30]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=0,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 1.0
Accuracy: 0.9900990099009901
Accuracy: 0.9900990099009901
Accuracy: 0.9801980198019802
Accuracy: 0.9603960396039604
Accuracy: 1.0
Accuracy: 0.9801980198019802
Accuracy: 0.9702970297029703
Accuracy: 0.9900990099009901
Accuracy: 0.9900990099009901
Unique authors: 101
Mean Acc: 0.985148514851485
Std Acc: 0.011922370870091393
Year: 2009
Accuracy: 0.9759615384615384
Accuracy: 0.9759615384615384
Accuracy: 0.9903846153846154
Accuracy: 0.9903846153846154
Accuracy: 0.9567307692307693
Accuracy: 0.9951923076923077
Accuracy: 0.9951923076923077
Accuracy: 0.9855769230769231
Accuracy: 0.9711538461538461
Accuracy: 0.9855769230769231
Unique authors: 208
Mean Acc: 0.9822115384615385
Std Acc: 0.011588433454994189
Year: 2010
Accuracy: 0.9868995633187773
Accuracy: 0.9868995633187773
Accuracy: 0.9912663755458515
Accuracy: 0.9912663755458515
Accuracy: 0.9912663755458515
Accuracy: 0.9956331877729258
Accuracy: 0.9694323144104804
Accuracy: 0.982532751091703
Accuracy: 0.9868995633187773


In [31]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=0,lang='py')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.9444444444444444
Accuracy: 0.9722222222222222
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Unique authors: 36
Mean Acc: 0.9916666666666668
Std Acc: 0.017786456215091253
Year: 2009
Accuracy: 0.9680851063829787
Accuracy: 0.9893617021276596
Accuracy: 0.9787234042553191
Accuracy: 1.0
Accuracy: 0.9787234042553191
Accuracy: 0.9893617021276596
Accuracy: 1.0
Accuracy: 0.9893617021276596
Accuracy: 0.9787234042553191
Accuracy: 0.9893617021276596
Unique authors: 94
Mean Acc: 0.9861702127659575
Std Acc: 0.009574468085106393
Year: 2010
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.9910714285714286
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.9910714285714286
Accuracy: 0.9910714285714286
Accuracy: 0.9910714285714286
Unique authors: 112
Mean Acc: 0.9964285714285716
Std Acc: 0.004374088826398517
Year: 2011
Accuracy: 1.0
Accuracy: 0.9957081545064378
Accuracy: 0.9957081545064378
Accuracy: 0.9914163

In [32]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=4,test_sample=1,obfuscated=0,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=4,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9029605263157895
Accuracy: 0.9046052631578947
Accuracy: 0.899671052631579
Accuracy: 0.9013157894736842
Accuracy: 0.8848684210526315
Accuracy: 0.8947368421052632
Accuracy: 0.8865131578947368
Accuracy: 0.8980263157894737
Accuracy: 0.8930921052631579
Accuracy: 0.9046052631578947
Unique authors: 608
Mean Acc: 0.8970394736842107
Std Acc: 0.006749435700224737
Year: 2009
Accuracy: 0.889873417721519
Accuracy: 0.8708860759493671
Accuracy: 0.9088607594936708
Accuracy: 0.8822784810126583
Accuracy: 0.8860759493670886
Accuracy: 0.8886075949367088
Accuracy: 0.8835443037974684
Accuracy: 0.9075949367088607
Accuracy: 0.8860759493670886
Accuracy: 0.9012658227848102
Unique authors: 790
Mean Acc: 0.8905063291139239
Std Acc: 0.011353661264046147
Year: 2010
Accuracy: 0.9473684210526315
Accuracy: 0.968421052631579
Accuracy: 0.952046783625731
Accuracy: 0.9555555555555556
Accuracy: 0.9391812865497076
Accuracy: 0.9461988304093567
Accuracy: 0.9426900584795321
Accuracy: 0.9497076023391813
A

In [33]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=6,test_sample=1,obfuscated=0,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=6,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9313725490196079
Accuracy: 0.954248366013072
Accuracy: 0.9379084967320261
Accuracy: 0.9411764705882353
Accuracy: 0.954248366013072
Accuracy: 0.9509803921568627
Accuracy: 0.9575163398692811
Accuracy: 0.9379084967320261
Accuracy: 0.934640522875817
Accuracy: 0.9379084967320261
Unique authors: 306
Mean Acc: 0.9437908496732026
Std Acc: 0.00898544253912911
Year: 2009
Accuracy: 0.9414141414141414
Accuracy: 0.9595959595959596
Accuracy: 0.9191919191919192
Accuracy: 0.9474747474747475
Accuracy: 0.9515151515151515
Accuracy: 0.9191919191919192
Accuracy: 0.9373737373737374
Accuracy: 0.9373737373737374
Accuracy: 0.9454545454545454
Accuracy: 0.9434343434343434
Unique authors: 495
Mean Acc: 0.9402020202020204
Std Acc: 0.012228481576098207
Year: 2010
Accuracy: 0.9803921568627451
Accuracy: 0.9750445632798574
Accuracy: 0.9786096256684492
Accuracy: 0.9786096256684492
Accuracy: 0.983957219251337
Accuracy: 0.9714795008912656
Accuracy: 0.9696969696969697
Accuracy: 0.983957219251337
Acc

In [34]:
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
for year in years:
    print("Year:",year)
    acc=[]
    for i in range(1,11):
        X_train,y_train,X_test,y_test=get_train_test(X,y,df,year,train_sample=8,test_sample=1,obfuscated=0,lang='java')
        neigh = KNeighborsClassifier(n_neighbors=8,p=1,n_jobs=-1,weights='distance')
        neigh.fit(X_train, y_train)
        y_probas=neigh.predict_proba(X_test)
        acc_score=top_k_accuracy(y_probas,y_test,neigh,3)
        print("Accuracy:",acc_score)
        acc.append(acc_score)
    print("Unique authors:",len(y_test))    
    print("Mean Acc:",np.mean(acc))
    print("Std Acc:",np.std(acc))

Year: 2008
Accuracy: 0.9653179190751445
Accuracy: 0.9653179190751445
Accuracy: 0.976878612716763
Accuracy: 0.953757225433526
Accuracy: 0.976878612716763
Accuracy: 0.9595375722543352
Accuracy: 0.976878612716763
Accuracy: 0.9710982658959537
Accuracy: 0.9710982658959537
Accuracy: 0.9826589595375722
Unique authors: 173
Mean Acc: 0.9699421965317919
Std Acc: 0.008495340148380957
Year: 2009
Accuracy: 0.95
Accuracy: 0.9785714285714285
Accuracy: 0.9785714285714285
Accuracy: 0.9678571428571429
Accuracy: 0.9642857142857143
Accuracy: 0.9571428571428572
Accuracy: 0.9642857142857143
Accuracy: 0.9535714285714286
Accuracy: 0.9607142857142857
Accuracy: 0.9642857142857143
Unique authors: 280
Mean Acc: 0.9639285714285715
Std Acc: 0.008957097288560308
Year: 2010
Accuracy: 0.990990990990991
Accuracy: 0.990990990990991
Accuracy: 0.987987987987988
Accuracy: 0.984984984984985
Accuracy: 0.993993993993994
Accuracy: 0.984984984984985
Accuracy: 0.975975975975976
Accuracy: 0.978978978978979
Accuracy: 0.99099099099